(03:How-to-package-a-Python)=
# How to package a Python
<hr>

To start this book, we will first develop an entire example Python package from beginning to end. This chapter forms the foundation of this book and aims to clearly and simply demonstrate the key steps involved in developing a Python package. Later chapters will explore each of these steps in more detail.

## partypy: simulate attendance at your party!

The example package we are going to create in this chapter will help us simulate guest attendance at an event - we'll call it `partpy`. Have you ever planned a party, a wedding, a conference, or any other kind of event and wondered how many of the invited guests will actually show up? Knowing this can be helpful for organizing things like seating and catering.

One way we can estimate how many guests will attend an event is by using simulations. The general idea is to assign a "probability of attendance" to each invited guest and then run a simulation of the event where we model each guest's attendance using the probability we assigned them. We can repeat this process as many times as we like to generate multiple estimates of how many guests might attend our real-life event.

We'll explore this concept further as we progress through the chapter, but below is an example of what the `partpy` package we are going to build can do. We'll first load in an example guest list of 100 guests, each with their own probability of attendance, using the `pandas` package and a csv file that is available in this book's [GitHub repository](https://github.com/UBC-MDS/py-pkgs/blob/master/docs/toy-data/party.csv):

```{note}
For this book, we assume readers have basic familiarity with the popular Python packages `numpy` and `pandas`.
```

```{prompt} python >>> auto
>>> import pandas as pd
>>> guest_list = pd.read_csv("https://raw.githubusercontent.com/UBC-MDS/py-pkgs/master/docs/toy-data/party.csv")
>>> guest_list
```

```python
               name  probability_of_attendance
0    Donovan Willis                       0.70
1   Jocelyn Navarro                       0.70
2     Houston Stein                       0.90
3    Carlos Mullins                       0.50
4    Bridger Pruitt                       0.70
..              ...                        ...
95   Maddox Santana                       0.50
96    Ariel Proctor                       0.50
97       Pedro Hull                       0.90
98  Janessa Collins                       0.95
99   Kendrick Burke                       0.30
```

We'll now use `partypy` to run 500 simulations of a party and plot the results in a histogram:

```{prompt} python >>> auto
>>> from partypy.simulate import simulate_party
>>> from partypy.plotting import plot_simulation
>>> results = simulate_party(p=guest_list["probability_of_attendance"], n_simulations=500)
>>> print(f"Average guests: {results["Total guests"].mean()}")
```

```console
68.81
```


```{prompt} python >>> auto
>>> plot_simulation(results)
```

```{figure} images/altair-plot-1.png
---
width: 60%
name: 03-altair-plot-1a
alt: Histogram of simulation results.
---
Histogram of simulation results.
```

## Package structure

The first thing we need to do to develop our `partypy` package is create an appropriate directory structure. Without getting too technical, a Python package is just a collection of Python modules. A module is a file with a *.py* extension that contains Python definitions and statements such as functions, classes, variables, and/or executable statements. The code you wish to easily reuse and potentially share with others will be contained within your package's modules. Along with these Python modules, packages typically include additional files such as documentation and tests that together, define a self-contained, shareable, and interpretable piece of software.

We'll discuss modules and Python package structure in detail in **Chapter 4: {ref}`04:Package-structure-and-distribution`**. While you can create your Python package structure from scratch if you know what you're doing, it's typically easier to use a pre-made template to set up your package structure and that's what we'll do here. We will use the Python package `cookiecutter` (which you installed back in **Chapter 2: {ref}`02:System-setup`**) to quickly create our package structure for us.

The `cookiecutter` package is a tool for populating a file and directory structure from a pre-made template. People have developed and open-sourced many different `cookiecutter` templates for different projects, such as for creating Python packages, R packages, websites, etc. You can find these templates by, for example, searching online repositories on [GitHub](https://www.github.com). We have developed our own `cookiecutter` [template](https://github.com/UBC-MDS/cookiecutter-ubc-mds) for creating Python packages to supplement this book. To use the `cookiecutter` template to set up the structure of our Python package, open up a terminal, change into the directory where you want your package to live and run the line of code below:

```{prompt} bash \$ auto
$ cookiecutter https://github.com/UBC-MDS/cookiecutter-ubc-mds.git
```

You will be prompted to provide information that will help customize the project and pre-populate files with information. Below is an example of how to respond to these prompts (default values for each attribute are shown in square brackets and hitting enter without entering any text will accept the default value). In this tutorial we will be calling our package `partypy`, however, we will eventually be publishing our package to Python's main package index [PyPI](https://pypi.org/). Package names on PyPI must be unique. As a result, **if you plan to follow along with this tutorial you should choose a unique name for your package**. Something like `partypy_[your intials]` might be appropriate, but you can always check if a particular name is already taken by visiting PyPI and searching for that name.

```console
author_name [Monty Python]: Tomas Beuzen
github_username [mpython]: TomasBeuzen
project_name [My Python package]: partypy
project_slug [partypy]: 
project_short_description [A package for doing great things!]: Simulate attendance at your party!
version [0.1.0]: 
python_version [3.9]: 
Select open_source_license:
1 - MIT
2 - Apache License 2.0
3 - GNU General Public License v3.0
4 - Creative Commons Attribution 4.0
5 - None
Choose from 1, 2, 3, 4, 5 [1]: 
Select include_github_actions:
1 - no
2 - build
3 - build+deploy
Choose from 1, 2, 3 [1]: 
```

```{attention}
Most of the options above are fairly self-explanatory but you'll learn more about each one as you make your way through this book. If this is your first time, just follow our lead above but be sure to use your own `author_name`, `github_username` and unique package name, such as `partypy_[your intials]`.

It's worth noting that in the example above we chose not to include any GitHub Actions files in our initial directory structure. GitHub Actions can help automate the building, testing and deployment of your Python package. We'll explore these topics in more detail in **Chapter 8: {ref}`08:Continuous-integration-and-deployment`**.
```

After responding to the `cookiecutter` prompts, we now have a new directory called `partypy`, with the following structure:

```
partypy
├── .gitignore
├── .readthedocs.yml
├── CHANGELOG.md
├── CONDUCT.md
├── CONTRIBUTING.md
├── docs
│   ├── make.bat
│   ├── Makefile
│   ├── requirements.txt
│   └── source
│       ├── changelog.md
│       ├── conduct.md
│       ├── conf.py
│       ├── contributing.md
│       ├── index.md
│       ├── installation.md
│       └── usage.ipynb
├── LICENSE
├── pyproject.toml
├── README.md
├── src
│   └── partypy
│       ├── __init__.py
│       └── partypy.py
└── tests
    └── test_partypy.py
```

This simple step has given us a file and directory structure suitable for building a fully-featured Python package. While there are quite a few files here, at this point we only need to worry about a few of these to get a working package together (we'll explore the others in later chapters). Specifically, we'll be working on:

1. `pyproject.toml`: the file that defines our project's metadata, dependencies, and how it will be built for distribution;
2. `src/`: the directory where we will write the Python source code (*.py* files) for our package;
3. `tests/`: the directory where we will write tests to ensure that our package's functions work as we expect; and,
4. `docs/`: the directory where we will write and build documentation for our package.

## Putting your project under version control

Before continuing to develop our package it is generally good practice to put your projects under local and remote version control, to better track changes to the project over time and to facilitate collaboration (if desired). The tools we recommend using for this are Git & GitHub (which we set up in **Chapter 2: {ref}`02:System-setup`)**. 

```{note}
For this book, we assume readers have [basic Git skills](https://git-scm.com/book/en/v2/Git-Basics-Getting-a-Git-Repository).
```

### Set up local version control

To set up local version control from a terminal, enter the root `partypy` directory, and initialize the project as a repository to be tracked by Git using:

```{prompt} bash \$ auto
$ cd partypy
$ git init
```

```console
Initialized empty Git repository in /Users/tomasbeuzen/GitHub/py-pkgs/partypy/.git/
```

Next, we need to tell Git which files to track (which will be all of them at this point) and then commit these changes locally:

```{prompt} bash \$ auto
$ git add .
$ git commit -m "initial package setup"
```

```console
[master (root-commit) 8b4edcb] initial package setup
 21 files changed, 601 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 .readthedocs.yml
 create mode 100755 CONDUCT.md
 ...
 create mode 100644 src/partypy/__init__.py
 create mode 100644 src/partypy/partypy.py
 create mode 100644 tests/test_partypy.py
```

### Set up remote version control

Now that we have set up our local version control, let's create a repository on [GitHub](https://github.com/) and set that as the remote version control home for this project. Head over to [GitHub](https://www.github.com) and create a new repository as demonstrated in the image below:

```{figure} images/set-up-github-1.png
---
width: 100%
name: 03-set-up-github-1
alt: Creating a new repository in GitHub.
---
Creating a new repository in GitHub.
```

To follow along with this tutorial, select the following options when setting up your GitHub repository: 

1. Give the GitHub repository the same name as your Python package and give it a short description;
2. Make the GitHub repository public; and,
3. Do not initialize the GitHub.com repository with any files (we've already created the files we need using our `cookiecutter` template).

```{figure} images/set-up-github-2.png
---
width: 100%
name: 03-github-2
alt: Setting up a new repository in GitHub.
---
Setting up a new repository in GitHub.
```

Next, copy the remote link to your repository and then use the commands below to link your local repository with the remote repository, and push your project to GitHub:

```{figure} images/set-up-github-3.png
---
width: 100%
name: 03-github-3
alt: Instructions on how to link local and remote repositories.
---
Instructions on how to link local and remote repositories.
```

```{prompt} bash \$ auto
$ git remote add origin git@github.com:TomasBeuzen/partypy.git
$ git branch -M main
$ git push -u origin main
```

```console
Enumerating objects: 28, done.
Counting objects: 100% (28/28), done.
Delta compression using up to 8 threads
Compressing objects: 100% (20/20), done.
Writing objects: 100% (28/28), 8.77 KiB | 2.19 MiB/s, done.
Total 28 (delta 0), reused 0 (delta 0)
To github.com:TomasBeuzen/partypy.git
 * [new branch]      main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.
```

```{attention}
To be clear, the commands above should be specific to your GitHub username and the name of your Python package. The example above uses SSH authentication with GitHub which we recommend setting up. SSH is useful for connecting to GitHub without having to supply your username and password every time. If you're interested in setting up SSH, take a look at the [GitHub documentation](https://docs.github.com/en/github/authenticating-to-github/connecting-to-github-with-ssh). If you don't have SSH authentication set up, HTTPS authentication works as well and would require the use of the following url in place of the one shown above to set the remote: `https://github.com/TomasBeuzen/partypy.git`. 
```

## Creating a virtual environment

Before we get started writing the Python code for our package, it is good practice to set up a virtual environment for our project. A virtual environment will help isolate our package and its dependencies from other software installed on our computer. There are several options available when it comes to creating and managing virtual environments (e.g., `conda`, `virtualenv`, etc.). We will use `conda` (which we installed back in **Chapter 2: {ref}`02:System-setup`**)) because it is a simple, commonly-used, and effective tool for managing virtual environments.

To use `conda` to create and activate a new virtual environment called `partypy` that includes Python 3.9, run the following in your terminal:

```{prompt} bash \$ auto
$ conda create --name partypy python=3.9 -y
```

To use this new environment for developing and installing software, we should "activate" the environment:

```{prompt} bash \$ auto
$ conda activate partypy
```

In most terminals, `conda` will by default add a prefix like `(partypy)` to your terminal prompt to indicate which environment you are working in. Anytime you wish to work on this package, you should activate this environment using the command above.

## Adding dependencies

Let's review the steps we've taken so far:
1. Set up our Python package structure using `cookiecutter`;
2. Put our project under local and remote version control using `git` and GitHub; and,
3. Created a virtual environment called `partypy` for our project.

We're now ready to start writing code and building our package. Those with some experience with Python packages will be aware of `setuptools` which is the traditional library used for building Python packages. However for beginners, `setuptools` has, in our opinion, a reasonably steep and unintuitive learning curve and can pose a barrier to entry into the packaging world. Luckily, there are many modern packaging tools now available that lower this barrier and help streamline the packaging process. The packaging tool we'll be using in this book to help build packages is `poetry`, which we mentioned and installed in **Chapter 2: {ref}`02:System-setup`**. As you'll see throughout this book, `poetry` is a simple, intuitive, and efficient tool for building and distributing Python packages and managing your package's dependencies.

Speaking of dependencies, often you'll know what other packages your package will depend on before even writing any code. For example, our `partypy` package is going to leverage the `numpy` and `pandas` packages. Thus, before we get started we should install these dependencies and record them as part of our package's metadata. It's fine if you don't know what dependencies your package will have in advance, you will be able to add new dependencies as you need them using the same workflow shown below (we'll also do this later in the chapter). We will use the command `poetry add` to add the `numpy` and `pandas` dependencies to our package. This command will install packages into the current environment and update the `[tool.poetry.dependencies]` section of the *`pyproject.toml`* file which currently only lists Python as a project dependency:

```toml
[tool.poetry.dependencies]
python = "^3.9"
```

Let's add `numpy` and `pandas` as dependencies now by running the following in the terminal:

```{prompt} bash \$ auto
$ poetry add numpy pandas
```

```console
Using version ^1.21.1 for numpy
Using version ^1.3.1 for pandas

Updating dependencies
Resolving dependencies... (6.4s)

Writing lock file

Package operations: 5 installs, 0 updates, 0 removals

  • Installing six (1.16.0)
  • Installing numpy (1.21.1)
  • Installing python-dateutil (2.8.2)
  • Installing pytz (2021.1)
  • Installing pandas (1.3.1)
```

Now if we view our *`pyproject.toml`* file we see that `numpy` and `pandas` are listed as a dependencies:

```toml
[tool.poetry.dependencies]
python = "^3.9"
numpy = "^1.21.1"
pandas = "^1.3.1"
```

```{tip}
The caret (^) prefixing the version numbers above indicates that our project depends on a version of the library that does not change the leftmost non-zero section (which indicates a major version change). For example, our package could use `numpy` 1.21.1, 1.21.2, or 1.25.3, but not 2.1.3. We'll talk more about why this is and what the numbers above mean in **Chapter 7: {ref}`07:Releasing-and-versioning`**.

For readers who have used *`requirements.txt`* before with `pip` or *`environment.yaml`* with `conda`, you can think of *`poetry.lock`* as the `poetry` equivalent of those files. As new versions of your specified dependencies become available, you can update your installed versions of them, within the constraints specified in *`pyproject.toml`*, by running `poetry update`.
```

Running `poetry add` actually changed two files, *`pyproject.toml`* (which we showed above and which records the metadata of our project) and *`poetry.lock`* (a record of all the packages, even the dependencies of our package's dependencies, and exact versions of them that `poetry` downloaded for this project).

The changes we've made above are important for our package, so let's commit them to local and remote version control:

```{prompt} bash \$ auto
$ git add pyproject.toml poetry.lock
$ git commit -m "build: add numpy and pandas as dependencies"
$ git push
```

```{tip}
Different developers use different syntax and formats when using a version control system. Here we take inspiration for our commit messages from the [Angular style guidelines](https://github.com/angular/angular.js/blob/master/DEVELOPERS.md#-git-commit-guidelines), which take the basic form "type: subject", where "type" indicates the kind of change being made and "subject" contains a description of the change. In this book, we'll frequently use the follow types:
- "build": indicates a change to the build system or external dependencies.
- "docs": indicates a change to documentation.
- "feat": indicates a new feature being added to the code base.
- "fix": indicates a bug fix.
- "test": indicates changes to testing framework.

We'll talk more about versioning in **Chapter 7: {ref}`07:Releasing-and-versioning`** and **Chapter 8: {ref}`08:Continuous-integration-and-deployment`**.
```

## Your first package code

We're now ready to write some Python code for our package! Recall that the package we want to create will estimate guest attendance at a party using simulations. The core idea is to assign a "probability of attendance" to each guest invited to the party and then simulate their attendance as a Bernoulli random variable. You can think of this as modelling each guest's attendance by flipping a coin with two sides, "won't attend" and "attend", but we can specify the probability of the coin landing on "attend". We can also flip the coin as many times as we like (i.e., run as many simulations as we like).

We can run a Bernoulli simulation using the `binomial` function in the `numpy` library, with the argument `n=1` (for the statistically inclined, a Bernoulli random variable is the same as a Binomial random variable with a single trial). As an example, imagine we have a guest that we believe will attend our party with a probability of 0.9 (90%). We can simulate the attendance of that guest by first opening up an interactive Python interpreter:

```{prompt} bash \$ auto
$ python
```

Then running the following code:

```{prompt} python >>> auto
>>> import numpy as np
>>> np.random.binomial(n=1, p=0.9)
```

```python
1
```

A value of `1` indicates the guest attended the party and a value of `0` indicates the guest did not attend the party. If you run the above code several times, you will see many `1`'s and a `0` every now and then. Rather than just re-running our code, we can repeat our simulation more efficiently using the `size` argument of the `binomial()` function. Let's run it 10 times:

```{prompt} python >>> auto
>>> simulations = 10
>>> probability = 0.9
>>> results = np.random.binomial(n=1, p=probability, size=simulations)
>>> results
```

```python
array([1, 1, 0, 1, 0, 0, 1, 0, 1, 1])
```

So our guest attended six of our simulated parties (there are six `1`'s). Now imagine we have three guests that we believe will attend our party with probabilities 0.3, 0.5, 0.9. We can simulate each guest's attendance in 10 simulations using the following code:

```{prompt} python >>> auto
>>> probability = [0.3, 0.5, 0.9]
>>> results = np.random.binomial(n=1, p=probability, size=(simulations, len(probability)))
>>> results
```

```python
array([[0, 1, 1],
       [0, 1, 0],
       [1, 0, 1],
       [0, 1, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 1],
       [0, 1, 1],
       [1, 1, 1],
       [1, 0, 1]])
```

The above array represents 10 simulations of three guests invited to a party. We typically want to know how many total guests attended each simulated party, so we should take the sum of each simulation:

```{prompt} python >>> auto
>>> results.sum(axis=1)
```

```python
array([2, 1, 2, 2, 1, 1, 2, 2, 3, 2])
```

It would be nice to display this information in a clean, tabular format. We can use a `pandas` DataFrame for that:

```{prompt} python >>> auto
>>> import pandas as pd
>>> (pd.DataFrame({"Total guests": results.sum(axis=1),
                   "Simulation": range(1, simulations + 1)})
       .set_index("Simulation")
    )
```

```python
            Total guests
Simulation              
1                      2
2                      1
3                      2
4                      2
5                      1
6                      1
7                      2
8                      2
9                      3
10                     2
```

We now have a nice way to run simulations of guest attendance at a party! But we don't want to have to re-run all that code every time we want to run some simulations. Let's turn the code into a function called `simulate_party()` and execute it in our interactive Python session:

```{note}
This book assumes you know how to write and document functions in Python. To learn more about this see [Think Python, Chapter 3: Functions](http://greenteapress.com/thinkpython/html/thinkpython004.html) by Allen Downey.
```

```{prompt} python >>> auto
>>> def simulate_party(p, n_simulations=500):
        """Simulate guest attendance at a party.

        The attendance of each guest is treated as a Bernoulli random variable
        with probability of attendance `p`. The total number of attending guests
        is summed up for each `n_simulations`.

        Parameters
        ----------
        p : float or array_like of floats
            Probability of guest attendance, >= 0 and <=1.
        n_simulations : int, optional
            Number of simulations to run. By default, 500.

        Returns
        -------
        pandas.DataFrame
            DataFrame with total number of guests per simulation.

        Examples
        --------
        >>> simulate_party([0.1, 0.5, 0.9], n_simulations=5)
                    Total guests
        Simulation
        1                      2
        2                      2
        3                      2
        4                      2
        5                      2
        """
        result = np.random.binomial(n=1, p=p, size=(n_simulations, len(p))).sum(axis=1)
        return pd.DataFrame(
            {"Total guests": result, "Simulation": range(1, n_simulations + 1)}
        ).set_index("Simulation")
```

We can now use the function as follows:

```{prompt} python >>> auto
>>> results = simulate_party(p=[0.3, 0.5, 0.9], n_simulations=10)
>>> results
```

```python
            Total guests
Simulation              
1                      2
2                      1
3                      2
4                      2
5                      1
6                      1
7                      2
8                      2
9                      3
10                     2
```

At this point, if you quit from the Python interactive session, the function we defined above will be lost and you will have to define it again in new sessions. The whole idea of a Python package is that we can store Python code, like our `simulate_party()` function, in an installable package that will allow us, or others, to reuse the code at will in any project without having to rewrite it.

So, let's now include the `simulate_party()` function into our `partypy` package. Where should we put it? Let's review the structure of our Python project:

```
partypy
├── .gitignore
├── .readthedocs.yml
├── CHANGELOG.md
├── CONDUCT.md
├── CONTRIBUTING.md
├── docs
├── LICENSE
├── pyproject.toml
├── README.md
├── src
│   └── partypy
│       ├── __init__.py
│       └── partypy.py
└── tests
```

All the code that we would like the user to run as part of our package should live inside the `src` directory. We'll discuss the layout of this package, including the `src` directory, more in **Chapter 4: {ref}`04:Package-structure-and-distribution`**. For a relatively small package with just a few functions, we typically put our code inside a single Python module (i.e., a `.py` file). Our template project directory structure already created and named such a module for us: `src/partypy/partypy.py`. Let's save our `simulate_party()` function there. Because our function depends on `numpy` and `pandas`, we should also be sure to import them at the top of the file. Here's what `src/partypy/partypy.py` should now look like:

```python
import numpy as np
import pandas as pd


def simulate_party(p, n_simulations=500):
    """Simulate guest attendance at a party.

    The attendance of each guest is treated as a Bernoulli random variable
    with probability of attendance `p`. The total number of attending guests
    is summed up for each `n_simulations`.

    Parameters
    ----------
    p : float or array_like of floats
        Probability of guest attendance, >= 0 and <=1.
    n_simulations : int, optional
        Number of simulations to run. By default, 500.

    Returns
    -------
    pandas.DataFrame
        DataFrame with total number of guests per simulation.

    Examples
    --------
    >>> simulate_party([0.1, 0.5, 0.9], n_simulations=5)
                Total guests
    Simulation
    1                      2
    2                      2
    3                      2
    4                      2
    5                      2
    """
    result = np.random.binomial(n=1, p=p, size=(n_simulations, len(p))).sum(axis=1)
    return pd.DataFrame(
        {"Total guests": result, "Simulation": range(1, n_simulations + 1)}
    ).set_index("Simulation")

```

## Test drive your package code

As stated earlier, the whole point of creating a package is so that we can easily reuse our code in any new Python project or interactive session. To test drive our `partypy` package, we can install it in our environment using `poetry install` from the root package directory:

```{prompt} bash \$ auto
$ poetry install
```

```console
Installing dependencies from lock file

No dependencies to install or update

Installing the current project: partypy (0.1.0)
```

```{note}
The above command will install `partypy` and its dependencies in the current virtual environment. Recall that we are working in the `partypy` environment which we activated by running `conda activate partypy` in the terminal.
```

Now, inside the root project directory we can open an interactive Python session:

```{prompt} bash \$ auto
$ python
```

Then import and use our `simulate_party` function from our `partypy` module with the following code:

```{prompt} python >>> auto
>>> from partypy.partypy import simulate_party
>>> simulate_party([0.1, 0.5, 0.9], n_simulations=5)
```

```python
            Total guests
Simulation              
1                      1
2                      2
3                      2
4                      2
5                      2
```

```{note}
The above syntax is telling Python to import the function `simulate_party()` from the `partypy` module of the `partypy` package. There are various other ways to import code from Python modules, which we'll explore more in **Chapter 4: {ref}`04:Package-structure-and-distribution`**.
```

Looks like everything is working! We now have a simple Python package that encapsulates the functionality we want to reuse. At this point, you've created a basic, functioning package which you could now use in different projects. In the next section, we'll add some additional code and functionality to our package. However, before we move on, it's important to note that `poetry install` installs your package in editable mode. This means that rather than installing an independent, built version of your package (something we'll talk more about in **Chapter 4: {ref}`04:Package-structure-and-distribution`**), it installs a link to your package's location. This is common practice for developers because it means that future edits you make to your package's source code are immediately available the next time you `import` it in a Python session, without having to `poetry install` again, as we'll see later in this chapter.

## Your second package code

For very simple packages, you may choose to add all your code into *`partypy.py`*. But more complex packages will benefit from better compartmentalisation and organisation of code into multiple, logical modules. To illustrate this point, we are going to add a plotting function to our `partypy` package which will plot a histogram of the simulation results output from the `simulate_party()` function. The code and workflow for creating a visualization is quite different to the simulation code we wrote previously, and it makes sense to create a new module to house the visualization code of our package.

To that end, we're now going to rename *`partypy.py`* to *`simulate.py`* and create a new module called *`plotting.py`* such that our package will now comprise two modules, each containing code for a distinct purpose:
1. *`src/partypy/plotting.py`*: contains code related to producing visualizations; and,
2. *`src/partypy/simulate.py`*: contains code related to running simulations.

With those changes, here's the structure of our Python project:

```
partypy
├── .gitignore
├── .readthedocs.yml
├── CHANGELOG.md
├── CONDUCT.md
├── CONTRIBUTING.md
├── docs
├── LICENSE
├── pyproject.toml
├── README.md
├── src
│   └── partypy
│       ├── __init__.py
│       ├── plotting.py
│       └── simulate.py
└── tests
```

We'll be using the `altair` library to make our visualization (but you could use any visualization library you like). Let's first add `altair` as a dependency of our package:

```{prompt} bash \$ auto
$ poetry add altair altair_viewer
```

Open up an interactive Python session, and try out the following code to produce a visualization. This example uses the example guest list available in this book's [GitHub repository](https://github.com/UBC-MDS/py-pkgs/blob/master/docs/toy-data/party.csv):

```{prompt} bash \$ auto
$ python
```

```{prompt} python >>> auto
>>> import pandas as pd
>>> guest_list = pd.read_csv("https://raw.githubusercontent.com/UBC-MDS/py-pkgs/master/docs/toy-data/party.csv")
>>> guest_list
```

```python
               Name  Probability of attendance
0    Donovan Willis                       0.70
1   Jocelyn Navarro                       0.70
2     Houston Stein                       0.90
3    Carlos Mullins                       0.50
4    Bridger Pruitt                       0.70
..              ...                        ...
95   Maddox Santana                       0.50
96    Ariel Proctor                       0.50
97       Pedro Hull                       0.90
98  Janessa Collins                       0.95
99   Kendrick Burke                       0.30
```

```{prompt} python >>> auto
>>> import altair as alt
>>> from partypy.simulate import simulate_party
>>> results = simulate_party(guest_list["probability_of_attendance"], n_simulations=500)
>>> histogram = (
        alt.Chart(results)
        .mark_bar()
        .encode(
            x=alt.X(
                "Total guests",
            ),
            y="count()",
            tooltip="count()",
        )
    )
>>> histogram.show()
```

```{figure} images/altair-plot-1.png
---
width: 50%
name: 03-altair-plot-1b
alt: Histogram of simulation results.
---
Histogram of simulation results.
```

To add this plotting functionality to our package, we can add the following code to *`plotting.py`*:

```python
import altair as alt


def plot_simulation(results):
    """Plot a histogram of simulation results.

    Parameters
    ----------
    results : pandas.DataFrame
        DataFrame of simulation results from `partpy.simulate_party()`

    Returns
    -------
    altair.Chart
        Histogram of simulation results.

    Examples
    --------
    >>> from partypy.simulate import simulate_party
    >>> from partypy.plotting import plot_simulation
    >>> results = simulate([0.1, 0.5, 0.9])
    >>> plot_simulation(results)
    altair.Chart
    """

    histogram = (
        alt.Chart(results)
        .mark_bar()
        .encode(
            x=alt.X(
                "Total guests",
                bin=alt.Bin(maxbins=30),
                axis=alt.Axis(format=".0f"),
            ),
            y="count()",
            tooltip="count()",
        )
    )

    return histogram

```

Let's make sure everything is working by trying out our new code in a fresh interactive Python session:

```{note}
Recall that when we installed our package with `poetry` using `poetry install`, it was installed in "editable mode", meaning that any changes to the source code will be immediately reflected when you next `import` the package, without the need to `poetry install` again.
```

```{prompt} bash \$ auto
$ python
```

Then import and use our package's functions with the following code:

```{prompt} python >>> auto
>>> from partypy.simulate import simulate_party
>>> from partypy.plotting import plot_simulation
>>> results = simulate_party([0.1, 0.5, 0.9], n_simulations=20)
>>> histogram = plot_simulation(results)
>>> histogram
```

```python
alt.Chart(...)
```

```{note}
Altair require a Javascript frontend to display charts. Notebook environments like Jupyter Notebook, JupyterLab, and Zeppelin combine a Python backend with a Javascript frontend, so can display Altair charts out-of-the-box. But when working in the Python interpreter from the command line, we need to explicitly call the `.show()` method which will display our chart in the browser: `histogram.show()`.
```

Now that we have a working package, you can exit your Python session and we should commit changes to local and remote version control. We'll use the shorthand `git add .` here to commit all our changed files to version control:

```{prompt} bash \$ auto
$ git add .
$ git commit -m "feat: add plotting module and refactor code"
$ git push
```

## Writing tests

At this point we have a package, `partypy`, which we can install locally in any environment and use in any project we wish. But to make our package robust and to ensure it does in fact do what it is supposed to do, we should write some formal unit tests. We'll discuss testing in detail in **Chapter 5: {ref}`05:Testing`**, but will go over the key steps here. In Python packages, tests typically live inside the *`tests`* directory, in files prefixed with *`test_*.py`*. For the `partypy` package, we have one such file called *`tests/test_partypy.py`*.

Let's add the below unit tests (`test_simulate_party()` and `test_plot_simulation()`) for our `partypy` package to *`tests/test_partypy.py`* now. These tests intend to test that our package is working as we expect it to. These tests typically check that a certain input results in a certain output. Take a look at the tests below and convince yourself that these tests are testing some of the expected behavior of our `partypy` package:

```python
from partypy.simulate import simulate_party
from partypy.plotting import plot_simulation
import pandas as pd
import altair as alt


def test_simulate_party():
    p_0 = [0, 0, 0]  # 3 guests with probability 0 of attending
    p_1 = [1]        # 1 guest with probability 1 of attending
    assert isinstance(simulate_party(p_0), pd.DataFrame)        # simulate_party outputs a dataframe
    assert simulate_party(p_0, 10)["Total guests"].sum() == 0   # 0 probability, 10 simulations gives 0 sum
    assert simulate_party(p_1, 10)["Total guests"].sum() == 10  # 1 probability, 10 simulations gives 10 sum


def test_plot_simulation():
    p_0 = [0, 0, 0]  # 3 guests with probability 0 of attending
    results = simulate_party(p_0)
    plot = plot_simulation(results)
    assert isinstance(plot, alt.Chart)  # plot_simulation outputs an alt.chart
    assert plot.mark == "bar"           # chart type is "bar"
    assert plot.data["Total guests"].sum() == 0  # 0 probability gives 0 sum
    
```

While we could run our test functions by starting a Python session, and importing and running them manually, it is much more efficient to automate the testing workflow. One way we can do this is to use the `pytest` package. A single call to `pytest` from the root of a project will look for all files of the form *`test_*.py`* or *`*_test.py`* and then execute all functions prefixed with *`test_*`*.

To try this out, we first add `pytest` as a development dependency via `poetry`:

```{prompt} bash \$ auto
$ poetry add --dev pytest
```

A development dependency is a package that is not required by a user to use your package, but is required for development purposes. The use of `--dev` in the above command specifies a development dependency. If you look in *`pyproject.toml`* you will see that `pytest` gets added under the `[tool.poetry.dev-dependencies]` section as opposed to the `[tool.poetry.dependencies]` section.

To run the above tests, we simply type the following in a terminal from our root package directory:

```{prompt} bash \$ auto
$ pytest
```

```console
============================= test session starts ==============================
platform darwin -- Python 3.9.6, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: /Users/tomasbeuzen/GitHub/py-pkgs/partypy
collected 2 items                                                              

tests/test_partypy.py ...                                                [100%]

============================== 2 passed in 0.29s ===============================
```

We get no error returned to us, indicating that our tests passed! This suggests that the code we wrote is correct (at least to our test specifications)! We'll explore writing tests in more detail in **Chapter 5: {ref}`05:Testing`**. For now, let's put our tests under local and remote version control:

```{prompt} bash \$ auto
$ git add pyproject.toml poetry.lock tests/test_partypy.py
$ git commit -m "test: add unit tests for partypy"
$ git push
```

## Package documentation

### Rendering documentation locally

For the users of your code (including your future self) it is important to have readable and accessible documentation describing how to install your package and how to use the code within it. We'll discuss documentation in detail in **Chapter 6: {ref}`06:Documentation`**, but for now, we will demonstrate the practical steps required to build Python package documentation. The aim here is to provide an overview and introduction to generating documentation, which you can build upon, if required, for future projects.

Let's review the structure of the `partypy` package we've been developing in this chapter:

```
partypy
├── .gitignore
├── .readthedocs.yml
├── CHANGELOG.md
├── CONDUCT.md
├── CONTRIBUTING.md
├── docs
│   ├── make.bat
│   ├── Makefile
│   ├── requirements.txt
│   └── source
│       ├── changelog.md
│       ├── conduct.md
│       ├── conf.py
│       ├── contributing.md
│       ├── index.md
│       ├── installation.md
│       └── usage.ipynb
├── LICENSE
├── pyproject.toml
├── README.md
├── src
└── tests
```

This is the most common setup for documenting a Python package and this boilerplate was created automatically for us with the `cookiecutter` template we used earlier in the chapter. It includes several key pieces of high-level documentation in the project's root directory: *`CHANGELOG.md`*, *`CONDUCT.md`*, *`CONTRIBUTING.md`*, *`LICENSE`*, and *`README.md`*. It also includes a *`docs`* folder which contains additional source and configuration files used to render your package's documentation into a format such as HTML or PDF with the help of a documentation generator tool. This is particularly important if you intend to share your package because it provides your package's documentation in an organized and accessible format that helps users explore and understand the functionality of your package without having to search through the source code or package directory structure. Depending on the intended audience of your package you may not need all of the documents shown in the structure above, as we'll talk more about in **Chapter 6: {ref}`06:Documentation`**.

Currently, the most commonly used tool in the Python packaging ecosystem for making documentation is `sphinx`. It is a "documentation generator" that translates a set of plain-text source files into various output formats (such as HTML or PDF).  Before we go about showing what "built documentation" looks like, let's briefly explore the contents of the `docs` folder which contains the source and configuration files that will be used to build our documentation:
- *`make.bat`* and *`Makefile`* contain commands needed to build our documentation with `sphinx` and do not need to be modified;
- *`requirement.txt`* contains documentation-specified dependencies that need to be defined when it comes to hosting our documentation online later in this chapter. It has been pre-populated by the `cookiecutter` template and does not need to be modified;
- The *`source`* directory contains the actual files that will make up our documentation. We'll explore each of these files in detail in **Chapter 6: {ref}`06:Documentation`** and not all these files will be required for every project. Some of these files simply contains relative references to documentation in our package's root - for example, *`docs/source/changelog.md`* simply contains a link to *`CHANGELOG.md`* - while others contains unique information such as how to install (*`installation.md`*) or use the package (*`usage.ipynb`*). The *.ipynb* extension indicates a Jupyter notebook file which contains code that will be executed when we build our documentation, with the output automatically rendered into the documentation, as we'll explore shortly. The *`source`* directory also contains a file *`conf.py`* which is a configuration file controlling how `sphinx` builds your documentation. You can read more about *`conf.py`* in the `sphinx` [documentation](https://www.sphinx-doc.org/en/master/usage/configuration.html) but for now, it has been pre-populated by the `cookiecutter` template and does not need to be modified.

At this point, you could edit your documentation files to add additional information about your package but for the purpose of this tutorial, we'll build our documentation as it is in its current state. To help us render all our individual documentation files into a single, coherent, easy-to-access document, we first need to install some new development dependencies:
- `myst-nb`: a packages required to help us render Markdown *.md* and Jupyter notebook *.ipynb* files into our documentation (we'll talk more about this a bit later);
- `sphinx-autoapi`: package that will help us extract docstrings from our code and render them into our documentation;
- `sphinx-rtd-theme`: a custom theme for styling the way our documentation will look; and,
- `sphinx-copybutton`: an extension that will add a helpful copy button to code snippets in our documentation.

```{prompt} bash \$ auto
$ poetry add --dev myst-nb sphinx-autoapi sphinx-rtd-theme sphinx-copybutton
```

It is typical to render documentation to *.html* for easy viewing and for sharing online. To do that, run the following:

```{prompt} bash \$ auto
$ make html -C docs
```

```console
Running Sphinx
making output directory... done
...
build succeeded.
The HTML pages are in _build/html.
```

```{attention}
To use `sphinx` extensions like we are doing here, we usually have to add them to the list `extensions = [...]` in the `conf.py` file in the `docs/source` directory. However, the `cookiecutter` template already took care of this for us.
```

If we now look inside our *`docs`* directory we see a new directory *`_build/html`* which contains the rendered *.html* files. We can open *`_build/html/index.html`* to view our documentation:

```{figure} images/documentation-1.png
---
width: 100%
name: 03-documentation-1
alt: The rendered documentation homepage.
---
The rendered documentation homepage.
```

The `sphinx-autoapi` extension extracted the docstrings within each module and rendered them into our documentation. You can find them by clicking "API Reference". For example, here are the functions and docstrings extracted from the `partypy.plotting` module (note there is currently only one function and docstring in this module):

```{figure} images/documentation-2.png
---
width: 100%
name: 03-documentation-2
alt: Documentation for the partypy plotting module.
---
Documentation for the partypy plotting module.
```

With our documentation nicely built, it's useful to briefly point out the utility of using the `myst-nb` extension to execute and render Jupyter notebook files (*.ipynb* extension) into the documentation. While this extension and workflow is not a necessity for building documentation, it provides the developer with a way of creating documentation that actually uses code from your package that gets executed as documentation is built. To illustrate this point, consider the raw *`docs/source/usage.ipynb`* file:

```{figure} images/documentation-4.png
---
width: 100%
name: 03-documentation-4
alt: Raw executable Jupyter notebook to be executed and rendered into partypy's documentation.
---
Raw executable Jupyter notebook to be executed and rendered into partypy's documentation.
```

Now look at the rendered page after building our documentation:

```{figure} images/documentation-5.png
---
width: 100%
name: 03-documentation-5
alt: Rendered Jupyter notebook document.
---
Rendered Jupyter notebook document.
```

Note how the code has been executed and the output is automatically shown in our documentation (the package version number is printed in this case). This is effective because it means the documentation and examples being shown to users are reproducible and any changes to the package will automatically be reflected in the documentation when you re-build it (i.e., you don't have to re-write the examples!).

Ultimately, you can easily and efficiently make beautiful and insightful documentation with `sphinx` and its ecosystem of extensions. We'll discuss this more in **Chapter 6: {ref}`06:Documentation`**, but for now let's commit our work to local and remote version control:

```{prompt} bash \$ auto
$ git add .
$ git commit -m "docs: created docs for partypy"
$ git push
```

### Rendering documentation online

If you intend to share your package with others, it will be useful to make your documentation accessible online. There are various ways to do this, but one of the most common and easiest ways is to link our GitHub repository to [Read the Docs](https://readthedocs.org/) - a service for automating the building, versioning, and hosting of documentation. To do this (at the time of writing):

1. Visit <https://readthedocs.org/> and click on "Sign up";
2. Select "Sign up with GitHub";
3. Click "Import a Project";
4. Click "Import Manually";
5. Fill in the project details by:
    1. Providing your package name (e.g., `partypy`);
    2. The GitHub repository URL (e.g., `https://github.com/TomasBeuzen/partypy`); and,
    3. Specify the default branch as `main`.
6. Click "Next" and then "Build version".

After following the steps above, your documentation should be successfully built by [Read the Docs](https://readthedocs.org/) and you should be able to access it via the "View Docs" button on the build page, or from the link that the `cookiecutter` template created for you at the top of the *`README.md`* file in your GitHub repository. For example, `partypy`'s documentation is now available at <https://partypy.readthedocs.io/en/latest/>. This documentation will be automatically re-built by Read the Docs each time you push changes to the specified default branch (`main` for us) of your GitHub repository.

```{note}
The *`.readthedocs.yml`* file that `cookiecutter` created for us in the root directory of our Python package contains some basic configuration settings for how Read the Docs should build our documentation. Importantly, this file tells Read the Docs that our documentation depends on packages specified in *`docs/requirements.txt`*.
```

## Commit and push to local and remote version control

We've now pushed a working version of our package to GitHub (assuming you chose to use GitHub to host a remote version of your project). In the next section we'll be building our package into a format suitable for distribution and then publishing it to PyPI. But before we get there, we should tag a release of our project's files on GitHub. Tagging a release means that we permanently mark a specific point in our repository's history which will always be available to anyone with access to your repository. In this case, we wish to identify the state of the repository at version 0.1.0 of `partypy`.

You can read more about tagging releases in the [GitHub documentation](https://docs.github.com/en/github/administering-a-repository/releasing-projects-on-github/about-releases), but it's as simple as going to the "Releases" tab of your GitHub repository and filling in some information about the release, as shown in the figure below. It's typical to identify the release with a “v” followed by the version of your package.

```{figure} images/release-0.1.0-meta.png
---
width: 100%
name: 03-release-0.1.0-meta
alt: Tagging a release of v0.1.0 of partypy on GitHub.
---
Tagging a release of v0.1.0 of partypy on GitHub.
```

Once the release is published, we will have a permanent record of the state of `partypy` at v0.1.0, which anyone with access to the repository can view and download.

```{figure} images/release-0.1.0.png
---
width: 100%
name: 03-release-0.1.0
alt: Tagged release of v0.1.0 of partypy on GitHub.
---
Tagged release of v0.1.0 of partypy on GitHub.
```

We'll talk more about making new versions and releases of your package as you update it (change source code, add features, fix bugs, etc.) in **Chapter 7: {ref}`07:Releasing-and-versioning`**.

## Building and distributing your package

### Building your package

Right now, our package is a collection of files and folders which we've been able to install ourselves with `poetry`, but that is difficult to share with others. So, the goal now is to bundle our package up into a distribution that can be easily shared and installed by users. This distribution includes the source code for your package, and can also include additional files such as documentation or tests. We'll talk much more about package structure and content in **Chapter 4: {ref}`04:Package-structure-and-distribution`**.

Modern packaging tools like `poetry` aim to make the process of creating distributions as seamless and accessible as possible, so that developers don't have to worry too much about the low-level details. The two main types of distributions are source distributions and built distributions. We'll talk about these in **Chapter 4: {ref}`04:Package-structure-and-distribution`**, but briefly:

- A source distribution is a single, compressed archive (e.g., *.tar.gz* or *.zip*) of the metadata and the source files needed to install your package. Source distributions require a build step before they can be installed. This build step coordinates building the contents of the package and its metadata into a form Python can `import`, as we'll talk more about in **Chapter 4: {ref}`04:Package-structure-and-distribution`**.
- A built distribution is a distribution containing source files and metadata that has been pre-built and does not require a build step before installation (unlike source distributions). The main built distribution format used by Python is the `wheel`, denoted with the extension *.whl*, and it is the preferred method of package distribution in Python.

A useful analogy for all this is to imagine making a cake for a party. A source distribution is a collection of all the individual ingredients (flour, eggs, butter, etc.) required to make the cake and requires time, expertise, and tools to make the cake. A built distribution is going to the store and buying a pre-made cake, all you need to do is bring it to the party! 

We can build the source and wheel distributions of our `partypy` package using `poetry build`:

```{prompt} bash \$ auto
$ poetry build
```

After running this command, you'll notice a new directory in your package called `dist`:

```
partypy
├── .gitignore
├── .readthedocs.yml
├── CHANGELOG.md
├── CONDUCT.md
├── CONTRIBUTING.md
├── dist
│   ├── partypy-0.1.0-py3-none-any.whl  <- built wheel distribution
│   └── partypy-0.1.0.tar.gz            <- source distribution
├── docs
├── LICENSE
├── pyproject.toml
├── README.md
├── src
└── tests
```

Those two new files are the source and built distributions of your package which can be easily distributed and installed by others. For example, to install the wheel (the preferred installation workflow), you could enter the following in a terminal:

```{prompt} bash \$ auto
$ pip install partypy-0.1.0-py3-none-any.whl
```

```console
Processing ./partypy-0.1.0-py3-none-any.whl
...
Successfully installed partypy-0.1.0
```

To install your package from source, you would have to unpack the source distribution archive, and could then run `pip install`:

```{prompt} bash \$ auto
$ gunzip -d partypy-0.1.0.tar.gz
$ cd partypy-0.1.0
$ pip install .
```

```console
Processing ./partypy-0.1.0-py3-none-any.whl
  Installing build dependencies ... done
    Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
...
Successfully built partypy
Successfully installed partypy-0.1.0
```

```{attention}
Note how installing from source requires a build step prior to installation.
```

Even if you only intend to use your package for personal projects, it can be useful to build and install distributions. Recall that when you install a package with `poetry` using `poetry install`, the package is installed in "editable mode", such that a link to the package's location is installed, rather than an independent distribution of the package itself. This is useful for development purposes, because it means that any changes to the source code will be immediately reflected when you `import` the package, without the need to `poetry install` again. However, for users of your package (potentially including yourself using your package in other projects), it is typically better to install a "non-editable" version of the package, which is the default behavior when you `pip install` a package. For example, you might wish to develop the package in one environment, but continue using a stable version of it in another. In such a case, you would `poetry install` in the first virtual environment, but `pip install` the wheel in the second environment.

### Publishing to TestPyPI

At this point, we have a distributions of `partypy` which we could use ourselves or share as we wish. If you do wish to share your package with the world, this is generally accomplished by sharing via the [Python Package Index (PyPI)](https://pypi.org/). However, it is good practice to do a "dry run" and check that everything works as expected by submitting to [TestPyPi](https://test.pypi.org/) first. `poetry` has a `publish` command which we can use to do this, however the default behavior is to publish to PyPI. So we need to add TestPyPI to the list of repositories `poetry` knows about via:

```{prompt} bash \$ auto
$ poetry config repositories.test-pypi https://test.pypi.org/legacy/
```

To publish to TestPyPI we can use `poetry publish` (you will be prompted for your TestPyPI username and password - sign up if you have not already done so):

```{prompt} bash \$ auto
$ poetry publish -r test-pypi
```

```console
Username: TomasBeuzen
Password: 
Publishing partypy (0.1.0) to test-pypi
 - Uploading partypy-0.1.0-py3-none-any.whl 100%
 - Uploading partypy-0.1.0.tar.gz 100%
```

```{tip}
It is recommended to use API tokens when uploading packages to PyPI rather than supplying a username and password as we did above. You can read more about that in the [PyPI documentation](https://pypi.org/help/#apitoken).
```

Now we should be able to visit our package on TestPyPI (for example, the URL for our package is: <https://test.pypi.org/project/partypy/>) and download it from there using `pip` via:

```{prompt} bash \$ auto
$ pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple partypy
```

```{attention}
By default `pip install` will search PyPI for the named package. However, we want to search testPyPI because that is where we uploaded our package. The argument `--index-url` points `pip` to the testPyPI index. However, our package `partypy` depends on some packages, like `pandas`, which can't be found on testPyPI (it is hosted only on PyPI). So, we need to use the `--extra-index-url` argument to also point `pip` to PyPI so that it can pull any necessary dependencies of `partypy` from there.
```

### Publishing to PyPI

If you're happy to officially share your package with the world, you can publish to PyPI by simply typing:

```{prompt} bash \$ auto
$ poetry publish
```

Your package will then be available on PyPI (e.g., <https://pypi.org/project/partypy/>) and can be installed with `pip`:

```{prompt} bash \$ auto
$ pip install partypy
```

## Summary and next steps

This chapter provided a practical overview of the key steps required to generate a fully-featured Python package. In the following chapters we'll explore and expand upon each of these steps in more detail. For those intending to share and collaborate on their package with others, a key workflow we have yet to discuss is continuous integration and continuous deployment (CI/CD) - that is, setting up automated pipelines for running tests, building documentation, and building and deploying your package. Such pipelines are an essential part of open source software and allow you to efficiently collaborate on software with others while maintaining package standards and functionality. We'll discuss CI/CD in **Chapter 8: {ref}`08:Continuous-integration-and-deployment`**.

Before moving onto the next chapter, let's summarize the steps we took for developing a Python package in this chapter:

1. Create package structure using a `cookiecutter` template:
    ```{prompt} bash \$ auto
    $ cookiecutter https://github.com/UBC-MDS/cookiecutter-ubc-mds.git
    ```
2. Create and activate a virtual environment using `conda`:
    ```{prompt} bash \$ auto
    $ conda create --name <your-env-name> python=3.9 -y
    $ conda activate <your-env-name>
    ```
3. Add package dependencies:
    ```{prompt} bash \$ auto
    $ poetry add <packages>
    ```
4. Write package code in the `src/` directory.
5. (Optional) Write tests in `tests/` directory. Add `pytest` as a development dependency, install package, and run tests:
    ```{prompt} bash \$ auto
    $ poetry add --dev pytest
    $ poetry install
    $ pytest
    ```
6. (Optional) Create documentation source files and render locally:
    ```{prompt} bash \$ auto
    $ make html -C docs
    ```
7. (Optional) Host documentation online with [Read the Docs](https://readthedocs.org/).
8. (Optional) Tag a release of your package on GitHub.
9. Build source and wheel distributions:
    ```{prompt} bash \$ auto
    $ poetry build
    ```
10. (Optional) Publish to [testPyPi](https://test.pypi.org/):
    ```{prompt} bash \$ auto
    $ poetry config repositories.test-pypi https://test.pypi.org/legacy/
    $ poetry publish -r test-pypi
    ```
11. (Optional) Install package from testPyPi to ensure everything is working as expected:
    ```{prompt} bash \$ auto
    pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple partypy
    ```
12. (Optional) Publish to [PyPi](https://test.pypi.org/):
    ```{prompt} bash \$ auto
    poetry publish
    ```
13. (Optional) Your package can now be installed by anyone:
    ```{prompt} bash \$ auto
    pip install <your-package-name>
    ```
    
The above workflow uses a particular suite of tools (e.g., `conda`, `poetry`, `sphinx`, etc.) to develop a Python package. While there are other tools that can be used to help build Python packages, the aim of this book is to give an intuitive and simple introduction to Python packaging using modern, popular tools, and this has influenced our selection of tools in this chapter and book. However, the concepts and workflow discussed here remain relevant to the Python packaging ecosystem, regardless of the tools you use to develop your Python packages.